In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from datetime import datetime

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import plot_partial_dependence
from loadDFRegion import getDF

import plotly.express as px

In [2]:
# Get adjusted dataset and list of dataset splitted by regions
df,dfRegionList = getDF()

Grabbing region 1 dataframe...
Grabbing region 2 dataframe...
Grabbing region 3 dataframe...
Grabbing region 4 dataframe...
Grabbing region 5 dataframe...
Grabbing region 6 dataframe...
Grabbing region 8 dataframe...
Grabbing region 9 dataframe...
Grabbing region 10 dataframe...


In [3]:
df.head()

,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,discovery_month,disc_date_pre,disc_pre_year,...,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness,day,month,year,stat_cause_encoded,Region
1,3.0,B,Arson,35.03833,-87.61000,TN,2006-12-11,Dec,11/11/2006,2006,...,59.8,8.4,0.0,86.8,0.184355,11,12,2006,1.0,8
3,1.0,B,Debris Burning,39.64140,-119.30830,NV,2005-06-06,Jun,5/7/2005,2005,...,10.4,7.2,0.0,0.0,0.487447,6,6,2005,2.0,4
7,1.0,B,Campfire,30.90472,-93.55750,TX,2005-11-12,Nov,10/13/2005,2005,...,28.4,27.5,1.2,55.4,0.241894,12,11,2005,4.0,8
8,1.0,B,Arson,35.90031,-92.06118,AR,2007-08-08,Aug,7/9/2007,2007,...,6.6,3.3,0.0,46.4,0.224629,8,8,2007,1.0,8
9,1.0,B,Miscellaneous,48.83940,-99.71850,ND,2004-04-19,Apr,3/20/2004,2004,...,12.3,1.8,0.0,0.0,0.291683,19,4,2004,3.0,1


In [4]:
# Print each region dataframe size
df_r1 = dfRegionList[0]
df_r2 = dfRegionList[1]
df_r3 = dfRegionList[2]
df_r4 = dfRegionList[3]
df_r5 = dfRegionList[4]
df_r6 = dfRegionList[5]
df_r8 = dfRegionList[6]
df_r9 = dfRegionList[7]
df_r10 = dfRegionList[8]
print("Setting up Notebook")
print("Total df size: " , df.size   )
for dftemp in dfRegionList:
    print("Size of region df: " , dftemp.size   )

Setting up Notebook
Total df size:  774408
Size of region df:  20049
Size of region df:  32226
Size of region df:  20254
Size of region df:  20500
Size of region df:  63919
Size of region df:  17015
Size of region df:  467933
Size of region df:  117998
Size of region df:  2542


In [5]:
def dropColumns(df):
    df = df.drop(columns=['fire_size_class', 'fire_mag', 'weather_file', 'state','stat_cause_descr'], axis=1)
    #print(df.size)
    df = df.drop(columns=['disc_clean_date', 'discovery_month', 'disc_date_pre', 'disc_pre_year', 'disc_pre_month'], axis=1) #, 'discovery_month', 'disc_date_pre', 'disc_pre_year' # drop date columns
    #print(df.size)
    df = df.drop(columns=['wstation_usaf', 'dstation_m', 'wstation_wban', 'wstation_byear', 'wstation_eyear'], axis=1) # drop station columns
    #print(df.size)
    return df

df = dropColumns(df)
print("After removing columns-Total df size: " , df.size   )
dfRegionList_new=[]
for dftemp in dfRegionList:
    dftemp = dropColumns(dftemp)
    print("Size of region df: " , dftemp.size   )
    dfRegionList_new.append(dftemp)
dfRegionList = dfRegionList_new

After removing columns-Total df size:  491088
Size of region df:  12714
Size of region df:  20436
Size of region df:  12844
Size of region df:  13000
Size of region df:  40534
Size of region df:  10790
Size of region df:  296738
Size of region df:  74828
Size of region df:  1612


In [6]:
def featureSelection(df,features):
    return df[features]


features =["latitude","remoteness","longitude","year","Temp_cont","Hum_pre_30","Wind_pre_30","fire_size","Prec_pre_30","month","Region","stat_cause_encoded"]
smallerDF= featureSelection(df,features)
print("After feature selection-Total df size: " , smallerDF.size   )
dfRegionList_small=[]
for dftemp in dfRegionList:
    dftemp = featureSelection(dftemp,features)
    print("Size of region df: " , dftemp.size   )
    dfRegionList_small.append(dftemp)


After feature selection-Total df size:  226656
Size of region df:  5868
Size of region df:  9432
Size of region df:  5928
Size of region df:  6000
Size of region df:  18708
Size of region df:  4980
Size of region df:  136956
Size of region df:  34536
Size of region df:  744


In [7]:
smallerDF.head()

,latitude,remoteness,longitude,year,Temp_cont,Hum_pre_30,Wind_pre_30,fire_size,Prec_pre_30,month,Region,stat_cause_encoded
1,35.03833,0.184355,-87.61000,2006,10.448298,70.840000,2.709764,3.0,59.8,12,8,1.0
3,39.64140,0.487447,-119.30830,2005,17.804904,44.778429,4.054982,1.0,10.4,6,4,2.0
7,30.90472,0.241894,-93.55750,2005,11.985560,72.899478,1.331257,1.0,28.4,11,8,4.0
8,35.90031,0.224629,-92.06118,2007,28.682688,68.319022,1.768074,1.0,6.6,8,8,1.0
9,48.83940,0.291683,-99.71850,2004,5.838721,64.606509,6.380760,1.0,12.3,4,1,3.0


In [8]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def runModel(df,model,title):
    X = df.drop(columns=['stat_cause_encoded'])
    Y = df['stat_cause_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=32)
    clf = model
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    #print(cm)
    acc=accuracy_score(y_test, y_pred)
    print('Accuracy of'+title +" "+ str(acc))

In [9]:
runModel(smallerDF,ExtraTreesClassifier(n_estimators=200, random_state=32)," entire df")
for i in range(len(dfRegionList_small)):
    regionNum= i+1
    if(regionNum>=7):
        regionNum+=1
    dftemp = dfRegionList_small[i]
    runModel(dftemp,ExtraTreesClassifier(n_estimators=200, random_state=32),f" region {regionNum}")

Accuracy of entire df 0.45738485971413445
Accuracy of region 1 0.5
Accuracy of region 2 0.46835443037974683
Accuracy of region 3 0.5858585858585859
Accuracy of region 4 0.45
Accuracy of region 5 0.34615384615384615
Accuracy of region 6 0.5060240963855421
Accuracy of region 8 0.46692947875602275
Accuracy of region 9 0.4236111111111111
Accuracy of region 10 0.6923076923076923
